In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
# Load data
file_path = "./Data file/usa_co2_temp_1800_2013.csv"
predict_temp_df = pd.read_csv(file_path)
predict_temp_df

country  year       co2  co2_per_capita  cement_co2  \
0    United States  1800     0.253           0.042         NaN   
1    United States  1801     0.267           0.044         NaN   
2    United States  1802     0.289           0.046         NaN   
3    United States  1803     0.297           0.047         NaN   
4    United States  1804     0.333           0.052         NaN   
..             ...   ...       ...             ...         ...   
209  United States  2009  5478.210          17.885      29.615   
210  United States  2010  5675.786          18.368      31.449   
211  United States  2011  5540.173          17.781      32.208   
212  United States  2012  5338.698          17.000      35.270   
213  United States  2013  5474.257          17.302      36.369   

     cement_co2_per_capita  coal_co2  coal_co2_per_capita  flaring_co2  \
0                      NaN     0.253                0.042          NaN   
1                      NaN     0.267                0.044          NaN   
2                      NaN     0.289                0.046          NaN   
3                      NaN     0.297                0.047          NaN   
4                      NaN     0.333                0.052          NaN   
..                     ...       ...                  ...          ...   
209                  0.097  1905.943                6.222       39.071   
210                  0.102  2013.304                6.515       41.343   
211                  0.103  1903.555                6.109       45.542   
212                  0.112  1684.037                5.362       48.883   
213                  0.115  1740.833                5.502       54.647   

      gas_co2  ...  co2_per_gdp  methane  methane_per_capita  nitrous_oxide  \
0         NaN  ...          NaN      NaN                 NaN            NaN   
1         NaN  ...          NaN      NaN                 NaN            NaN   
2         NaN  ...          NaN      NaN                 NaN            NaN   
3         NaN  ...          NaN      NaN                 NaN            NaN   
4         NaN  ...          NaN      NaN                 NaN            NaN   
..        ...  ...          ...      ...                 ...            ...   
209  1245.742  ...        0.368   649.89               2.122         241.81   
210  1308.832  ...        0.372   650.38               2.105         247.17   
211  1327.528  ...        0.357   628.73               2.018         255.55   
212  1388.255  ...        0.337   618.71               1.970         246.42   
213  1427.506  ...        0.340   619.02               1.956         247.39   

     nitrous_oxide_per_capita  population           gdp  energy_per_gdp  \
0                         NaN     6000000           NaN             NaN   
1                         NaN     6113782           NaN             NaN   
2                         NaN     6229723           NaN             NaN   
3                         NaN     6347862           NaN             NaN   
4                         NaN     6468241           NaN             NaN   
..                        ...         ...           ...             ...   
209                     0.789   306307565  1.489429e+13           1.676   
210                     0.800   309011469  1.527062e+13           1.688   
211                     0.820   311584051  1.550930e+13           1.650   
212                     0.785   314043885  1.584964e+13           1.567   
213                     0.782   316400539  1.610843e+13           1.589   

     Avg Temperature  Avg Temperature Uncertainty  
0          12.377868                     3.205628  
1          12.681863                     2.895417  
2          12.859703                     3.065703  
3          12.692603                     3.065032  
4          12.576432                     3.379515  
..               ...                          ...  
209        11.156327                     0.220114  
210        11.732877                     0.211351  
211        

In [4]:
# Removing the NaN values by replacing the NaN values with the mean average. 
predict_temp_df['flaring_co2'].fillna(predict_temp_df['flaring_co2'].mean(), inplace = True)
predict_temp_df['cement_co2'].fillna(predict_temp_df['cement_co2'].mean(), inplace = True)
predict_temp_df['cement_co2_per_capita'].fillna(predict_temp_df['cement_co2_per_capita'].mean(), inplace = True)
predict_temp_df['gas_co2'].fillna(predict_temp_df['gas_co2'].mean(), inplace = True)
predict_temp_df['gas_co2_per_capita'].fillna(predict_temp_df['gas_co2_per_capita'].mean(), inplace = True)
predict_temp_df['oil_co2'].fillna(predict_temp_df['oil_co2'].mean(), inplace = True)
predict_temp_df['oil_co2_per_capita'].fillna(predict_temp_df['oil_co2_per_capita'].mean(), inplace = True)
predict_temp_df['co2_per_gdp'].fillna(predict_temp_df['co2_per_gdp'].mean(), inplace = True)
predict_temp_df['methane'].fillna(predict_temp_df['methane'].mean(), inplace = True)
predict_temp_df['methane_per_capita'].fillna(predict_temp_df['methane_per_capita'].mean(), inplace = True)
predict_temp_df['nitrous_oxide'].fillna(predict_temp_df['nitrous_oxide'].mean(), inplace = True)
predict_temp_df['nitrous_oxide_per_capita'].fillna(predict_temp_df['nitrous_oxide_per_capita'].mean(), inplace = True)
predict_temp_df['gdp'].fillna(predict_temp_df['gdp'].mean(), inplace = True)
predict_temp_df['energy_per_gdp'].fillna(predict_temp_df['energy_per_gdp'].mean(), inplace = True)
predict_temp_df['co2_growth_prct'].fillna(predict_temp_df['co2_growth_prct'].mean(), inplace = True)

predict_temp_df

country  year       co2  co2_per_capita  cement_co2  \
0    United States  1800     0.253           0.042   18.398612   
1    United States  1801     0.267           0.044   18.398612   
2    United States  1802     0.289           0.046   18.398612   
3    United States  1803     0.297           0.047   18.398612   
4    United States  1804     0.333           0.052   18.398612   
..             ...   ...       ...             ...         ...   
209  United States  2009  5478.210          17.885   29.615000   
210  United States  2010  5675.786          18.368   31.449000   
211  United States  2011  5540.173          17.781   32.208000   
212  United States  2012  5338.698          17.000   35.270000   
213  United States  2013  5474.257          17.302   36.369000   

     cement_co2_per_capita  coal_co2  coal_co2_per_capita  flaring_co2  \
0                 0.085731     0.253                0.042    27.881063   
1                 0.085731     0.267                0.044    27.881063   
2                 0.085731     0.289                0.046    27.881063   
3                 0.085731     0.297                0.047    27.881063   
4                 0.085731     0.333                0.052    27.881063   
..                     ...       ...                  ...          ...   
209               0.097000  1905.943                6.222    39.071000   
210               0.102000  2013.304                6.515    41.343000   
211               0.103000  1903.555                6.109    45.542000   
212               0.112000  1684.037                5.362    48.883000   
213               0.115000  1740.833                5.502    54.647000   

         gas_co2  ...  co2_per_gdp   methane  methane_per_capita  \
0     497.198833  ...     0.718742  688.9625              2.4435   
1     497.198833  ...     0.718742  688.9625              2.4435   
2     497.198833  ...     0.718742  688.9625              2.4435   
3     497.198833  ...     0.718742  688.9625              2.4435   
4     497.198833  ...     0.718742  688.9625              2.4435   
..           ...  ...          ...       ...                 ...   
209  1245.742000  ...     0.368000  649.8900              2.1220   
210  1308.832000  ...     0.372000  650.3800              2.1050   
211  1327.528000  ...     0.357000  628.7300              2.0180   
212  1388.255000  ...     0.337000  618.7100              1.9700   
213  1427.506000  ...     0.340000  619.0200              1.9560   

     nitrous_oxide  nitrous_oxide_per_capita  population           gdp  \
0       259.413333                  0.916625     6000000  3.055051e+12   
1       259.413333                  0.916625     6113782  3.055051e+12   
2       259.413333                  0.916625     6229723  3.055051e+12   
3       259.413333                  0.916625     6347862  3.055051e+12   
4       259.413333                  0.916625     6468241  3.055051e+12   
..             ...                       ...         ...           ...   
209     241.810000                  0.789000   306307565  1.489429e+13   
210     247.170000                  0.800000   309011469  1.527062e+13   
211     255.550000                  0.820000   311584051  1.550930e+13   
212     246.420000                  0.785000   314043885  1.584964e+13   
213     247.390000                  0.782000   316400539  1.610843e+13   

     energy_per_gdp  Avg Temperature  Avg Temperature Uncertainty  
0          2.602592        12.377868                     3.205628  
1          2.602592        12.681863                     2.895417  
2          2.602592        12.859703                     3.065703  
3          2.602592        12.692603                     3.065032  
4          2.602592        12.576432                     3.379515  
..              ...              ...                          ...  
209        1.676000        11.156327                     0.220114  
210        1.688000        11.732877                     0.211351  
211        

In [5]:
columns = [
    'country', 'year', 'co2', 'co2_per_capita', 'cement_co2',
       'cement_co2_per_capita', 'coal_co2', 'coal_co2_per_capita',
       'flaring_co2', 'gas_co2', 'gas_co2_per_capita', 'oil_co2',
       'oil_co2_per_capita', 'co2_growth_prct', 'co2_per_gdp', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'population', 'gdp', 'energy_per_gdp',
       'Avg Temperature Uncertainty']

target = ['Avg Temperature']

In [13]:
# Create our features
X = predict_temp_df.drop(columns = "Avg Temperature")
X = pd.get_dummies(X, columns = ["country"])

# Create our target
y = predict_temp_df['Avg Temperature'].astype('int').to_frame()

In [14]:
X.describe()

year          co2  co2_per_capita  cement_co2  \
count   214.000000   214.000000      214.000000  214.000000   
mean   1906.500000  1775.834846        9.659154   18.398612   
std      61.920648  2020.307607        7.936626   12.609423   
min    1800.000000     0.253000        0.042000    0.000000   
25%    1853.250000    30.911250        1.185250   10.409500   
50%    1906.500000  1044.142000       10.775000   18.398612   
75%    1959.750000  2879.977000       16.341000   28.458000   
max    2013.000000  6134.521000       22.236000   46.851000   

       cement_co2_per_capita     coal_co2  coal_co2_per_capita  flaring_co2  \
count             214.000000   214.000000           214.000000   214.000000   
mean                0.085731   777.669472             5.111668    27.881063   
std                 0.052663   713.472753             3.865967     8.113161   
min                 0.000000     0.253000             0.042000     4.990000   
25%                 0.076500    30.911250             1.185250    27.881063   
50%                 0.085731   887.893500             5.586500    27.881063   
75%                 0.130500  1312.315750             7.463000    27.881063   
max                 0.171000  2214.838000            14.344000    54.647000   

           gas_co2  gas_co2_per_capita  ...  co2_per_gdp     methane  \
count   214.000000          214.000000  ...   214.000000  214.000000   
mean    497.198833            2.186538  ...     0.718742  688.962500   
std     387.022883            1.459075  ...     0.410101   16.864325   
min       0.165000            0.003000  ...     0.029000  618.710000   
25%      99.791250            0.769000  ...     0.399500  688.962500   
50%     497.198833            2.186538  ...     0.718742  688.962500   
75%     620.821250            3.324750  ...     0.993250  688.962500   
max    1427.506000            5.231000  ...     1.648000  770.750000   

       methane_per_capita  nitrous_oxide  nitrous_oxide_per_capita  \
count          214.000000     214.000000                214.000000   
mean             2.443500     259.413333                  0.916625   
std              0.117713       3.319460                  0.028872   
min              1.956000     241.810000                  0.782000   
25%              2.443500     259.413333                  0.916625   
50%              2.443500     259.413333                  0.916625   
75%              2.443500     259.413333                  0.916625   
max              3.044000     281.250000                  1.048000   

         population           gdp  energy_per_gdp  \
count  2.140000e+02  2.140000e+02      214.000000   
mean   1.119351e+08  3.055051e+12        2.602592   
std    9.337697e+07  4.122802e+12        0.337572   
min    6.000000e+06  2.668967e+10        1.567000   
25%    2.607410e+07  2.198610e+11        2.602592   
50%    8.967570e+07  1.237754e+12        2.602592   
75%    1.859870e+08  3.242571e+12        2.602592   
max    3.164005e+08  1.610843e+13        3.746000   

       Avg Temperature Uncertainty  country_United States  
count                   214.000000                  214.0  
mean                      1.052999                    1.0  
std                       1.088695                    0.0  
min                       0.169596                    1.0  
25%                       0.230978                    1.0  
50%                       0.389550                    1.0  
75%                       1.691417                    1.0  
max                       3.706145                    1.0  

[8 rows x 23 columns]

In [15]:
# Check the balance of our target values
y['Avg Temperature'].value_counts()

10    114
11     57
9      27
12     11
8       4
13      1
Name: Avg Temperature, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
ranforest_model = BalancedRandomForestClassifier(n_estimators=100, random_state = 1)
ranforest_model

BalancedRandomForestClassifier(random_state=1)

In [18]:
# Calculated the balanced accuracy score
ranforest_model = ranforest_model.fit(X_train, y_train)
y_pred = ranforest_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.43818681318681313

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 3,  0,  0,  0,  0,  0],
       [ 3,  4,  0,  0,  0,  0],
       [ 5,  7, 13,  1,  0,  0],
       [ 3,  1,  5,  4,  0,  0],
       [ 1,  0,  0,  2,  1,  0],
       [ 0,  0,  0,  1,  0,  0]])

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          8       0.20      1.00      0.76      0.33      0.87      0.78         3
          9       0.33      0.57      0.83      0.42      0.69      0.46         7
         10       0.72      0.50      0.82      0.59      0.64      0.40        26
         11       0.50      0.31      0.90      0.38      0.53      0.26        13
         12       1.00      0.25      1.00      0.40      0.50      0.23         4
         13       0.00      0.00      1.00      0.00      0.00      0.00         1

avg / total       0.60      0.46      0.86      0.48      0.61      0.37        54

